In [2]:
import unittest
import os
import pandas as pd
from video_quality_map import merge_data

class TestMergeData(unittest.TestCase):
    def setUp(self):
        self.feature_path = "/Users/khanhha/mpt-statistical-testing/test_MPTData/feature_data"
        self.msd_path = "/Users/khanhha/mpt-statistical-testing/test_MPTData/msd_data"
        self.json_path = "/Users/khanhha/mpt-statistical-testing/test_MPTData/15_models_10_percent.json"

    # Pass
    def test_basic_functionality(self):
        [df, msd_data, quality_data] = merge_data(self.feature_path, self.msd_path, self.json_path)
        self.assertIsInstance(df, dict)
        self.assertIsInstance(msd_data, dict)
        self.assertIsInstance(quality_data, dict)
    
    # Pass
    def test_input_validation(self):
        # Test with invalid paths
        with self.assertRaises(Exception):
            merge_data("invalid/path", self.msd_path, self.json_path)
            merge_data(self.feature_path, "invalid/path", self.json_path)
            merge_data(self.feature_path, self.msd_path, "invalid/path")

    # Pass
    def test_data_integrity(self):
        df, _, _ = merge_data(self.feature_path, self.msd_path, self.json_path)
        # Number of columns = 45
        for key in df.keys():
            self.assertEqual(df[key].shape[1], 45)

    # Pass
    def test_corner_cases(self):
        # Test with empty directories
        with self.assertRaises(Exception):
            merge_data("empty/feature/directory", self.msd_path, self.json_path)
            merge_data(self.feature_path, "empty/msd/directory", self.json_path)

    # Fail
    def test_expected_columns(self):
        df, _, _ = merge_data(self.feature_path, self.msd_path, self.json_path)
        expected_columns = ['Unnamed: 0', 'Frame', 'Gauss', 'MSDs', 'Mean_Intensity', 'Quality_x',
                            'SN_Ratio', 'Track_ID', 'X', 'Y', 'alpha', 'D_fit', 'kurtosis',
                            'asymmetry1', 'asymmetry2', 'asymmetry3', 'AR', 'elongation',
                            'boundedness', 'fractal_dim', 'trappedness', 'efficiency',
                            'straightness', 'MSD_ratio', 'Deff1', 'Deff2', 'Mean alpha',
                            'Mean D_fit', 'Mean kurtosis', 'Mean asymmetry1', 'Mean asymmetry2',
                            'Mean asymmetry3', 'Mean AR', 'Mean elongation', 'Mean boundedness',
                            'Mean fractal_dim', 'Mean trappedness', 'Mean efficiency',
                            'Mean straightness', 'Mean MSD_ratio', 'Mean Deff1', 'Mean Deff2',
                            'frames', 'Quality_y', 'Category']  # List of expected columns
        for key in df.keys():    
            self.assertCountEqual(list(df[key].columns), expected_columns)


# if __name__ == '__main__':
#     unittest.main()

suite = unittest.TestLoader().loadTestsFromTestCase(TestMergeData)
_ = unittest.TextTestRunner().run(suite)

.....
----------------------------------------------------------------------
Ran 5 tests in 77.738s

OK


In [51]:
import unittest
from unittest.mock import patch

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from video_quality_map import merge_data, trajectory_plot, zoom_trajectory_plot, distruibution_by_age
import random

class TestPlotFunctions(unittest.TestCase):
    def setUp(self):
        self.feature_path = "/Users/khanhha/mpt-statistical-testing/test_MPTData/feature_data"
        self.msd_path = "/Users/khanhha/mpt-statistical-testing/test_MPTData/msd_data"
        self.json_path = "/Users/khanhha/mpt-statistical-testing/test_MPTData/15_models_10_percent.json"
        self.vid_codes = []
        for f1 in os.listdir(self.feature_path):
            if f1.endswith('.csv'):

                # Extract the tail in feature file
                feature_tail = f1.split('_',1)[1].split('.')[0]
                
                # Iterate over each file in msd
                for f2 in os.listdir(self.msd_path):
                    
                    # Extract the tail in msd file
                    msd_tail = f2.split('_',1)[1].split('.')[0]
            
                    # If tails are the same
                    if feature_tail == msd_tail:
            
                        # Add tail into video_codes list
                        self.vid_codes.append(msd_tail)
        self.merge_df,_,_ = merge_data(self.feature_path, self.msd_path, self.json_path)
   
            
    # def test_trajectory_plot(self):
    #     vid_code = random.choice(self.vid_codes)
    #     with patch("video_quality_map.plt.show") as show_patch, \
    #         patch("video_quality_map.plt.title") as title_patch, \
    #         patch("video_quality_map.plt.legend") as legend_patch:

    #         trajectory_plot(self.merge_df, vid_code)
    #         # Test if the plot is called
    #         assert show_patch.called

    #         # Test if it is the right title
    #         title_patch.assert_called_once_with('Trajectories of Particles in Video ' + vid_code)        

    #         # Test if legend is called
    #         legend_patch.assert_called_once()


    # def test_zoom_trajectory_plot(self):
    #     vid_code = random.choice(self.vid_codes)
    #     df = self.merge_df[vid_code]
    #     x1 = random.randint(0, int(max(df['X'].dropna(), default = 0)))
    #     x2 = random.randint(0, int(max(df['X'].dropna(), default = 0)))
    #     y1 = random.randint(0, int(max(df['Y'].dropna(), default = 0)))
    #     y2 = random.randint(0, int(max(df['Y'].dropna(), default = 0)))

    #     with patch("video_quality_map.plt.show") as show_patch, \
    #         patch("video_quality_map.plt.title") as title_patch, \
    #         patch("video_quality_map.plt.legend") as legend_patch:

    #         zoom_trajectory_plot(self.merge_df, vid_code, x1, x2, y1, y2)
    #         # Test if the plot is called
    #         assert show_patch.called

    #         # Test if it is the right title
    #         title_patch.assert_called_once_with('Zoom Trajectories of Particles in Video ' + vid_code)        

    #         # Test if legend is called
    #         legend_patch.assert_called_once()

    def test_distruibution_by_age(self):
        _, _, quality_data = merge_data(self.feature_path, self.msd_path, self.json_path)
        
        with patch("video_quality_map.plt.show") as show_patch, \
            patch("video_quality_map.plt.title") as title_patch, \
            patch("video_quality_map.plt.grid") as grid_patch:
            # patch("video_quality_map.plt.xlabel") as xlabel_patch, \
            # patch("video_quality_map.plt.ylabel") as ylabel_patch:

            distruibution_by_age(self.feature_path, self.msd_path, quality_data)
            # Test if the plot is called
            assert show_patch.called

            # Test if it is the right title
            title_patch.assert_called_once_with('Mean Quality Score Distribution between Different Ages')      

            # Test if legend is called
            grid_patch.assert_called_once_with(True)

            # # Check x y label
            # xlabel_patch.assert_called_once_with('Mean Quality Score')
            # ylabel_patch.assert_called_once_with('Quality')
            

# if __name__ == '__main__':
#     unittest.main()

suite = unittest.TestLoader().loadTestsFromTestCase(TestPlotFunctions)
_ = unittest.TextTestRunner().run(suite)

E
ERROR: test_distruibution_by_age (__main__.TestPlotFunctions.test_distruibution_by_age)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/xw/ch0shqmj2sb_q32yc2lfnhfr0000gn/T/ipykernel_3168/2427295945.py", line 86, in test_distruibution_by_age
    distruibution_by_age(self.feature_path, self.msd_path, quality_data)
  File "/Users/khanhha/mpt-statistical-testing/Workspaces/video_quality_map.py", line 258, in distruibution_by_age
    for i,code in enumerate(vid_codes):
                  ^^^^^^^^^^^^^^^^^^^^^^
KeyError: 'features_Store'

----------------------------------------------------------------------
Ran 1 test in 90.426s

FAILED (errors=1)
